In [213]:
from __future__ import print_function
import aerospike
from aerospike import predicates
from aerospike import exception
import logging
import time

logging.getLogger().setLevel(logging.WARNING)

maxLoops = 100

aeroHost = ("127.0.0.1", 3000)
aeroSet = "customers"

In [214]:
# Configure the client
config = {
    'hosts': [ aeroHost ]
}

In [215]:
# Create a client and connect it to the cluster
try:
    client = aerospike.client(config).connect()
except:
    import sys
    print("failed to connect to the cluster with", config['hosts'])
    sys.exit(1)

In [216]:
# Выбор пространства имен и его предварительная инициализация 

# Поскольку предупреждали, что запуск будет производиться на чистом аэроспайке:
#   - мы не знаем, какой там неймспейс, в итоге берем первый попавшийся :)
#   - создаем вторичный индекс по бину телефона

aeroNameSpace = client.info_node("namespaces", aeroHost, None).split("\n")[0].split("\t")[1]
print("Cluster's current namespace is",aeroNameSpace)

try:
    print("Creating indexes...")
    client.index_string_create(aeroNameSpace, aeroSet, "phone_number", aeroNameSpace+"_"+aeroSet+"_phone_number_idx")
except exception.IndexFoundError:
    print("Needed indexes already exists")
else:
    print("Indexes created")

print("Secondary indexes:")
print(client.info('sindex/test'))

Cluster's current namespace is test
Creating indexes...
Needed indexes already exists
Secondary indexes:
{'BB9020015AC4202': (None, 'ns=test:set=customers:indexname=test_customers_phone_number_idx:bin=phone_number:type=STRING:indextype=NONE:path=phone_number:state=RW;\n')}


In [217]:
def add_customer(customer_id, phone_number:str, lifetime_value):
    try:
        client.put( (aeroNameSpace, aeroSet, customer_id), {
            'phone_number': phone_number,
            'lifetime_value': lifetime_value
         })
    except Exception as e:
        logger.error("error: {0}".format(e))

In [218]:
# Дополнительная функция нахождения любого клиентского бина по главному ключу
def get_bin_by_id(customer_id, binName):
    try:
        (key, metadata, record) = client.get((aeroNameSpace, aeroSet, customer_id))
        return record.get(binName)
    except Exception as e:
        logger.error('Requested non-existent customer ' + str(customer_id))

In [219]:
def get_ltv_by_id(customer_id):
    try:
        (key, metadata, record) = client.get((aeroNameSpace, aeroSet, customer_id))
        return record.get("lifetime_value")
    except Exception as e:
        logger.error('Requested non-existent customer ' + str(customer_id))

In [220]:
def get_ltv_by_phone(phone_number:str):
    try:
        # Создаем запрос для поиска телефона
        aeroQuery = client.query(aeroNameSpace, aeroSet)
        (key, metadata, bins) = client.select((aeroNameSpace, aeroSet, phone_number), ["lifetime_value"])
        return bins.get("lifetime_value")
    except Exception as e:
        logging.error('Requested phone number is not found ' + str(phone_number))

In [221]:
startTime = time.time()
for i in range(1, maxLoops):
    add_customer( i, i + 1, i + 2 )
print("Insert time by id: ", (time.time()-startTime))

Insert time by id:  0.04474639892578125


In [222]:
print ("All is OK! :)")

All is OK! :)


In [226]:
#    (key, meta, bins) = client.select(key, ['name'])
#aeroQuery.select("lifetime_value")
#aeroQuery.where(predicates.equals("phone_number", "11"))
#results = aeroQuery.results( {'total_timeout':2000})
#print(results)

print(get_ltv_by_id(1))
print("'"+str(get_bin_by_id(1,"phone_number"))+"'")
print(get_ltv_by_phone(21212))

ERROR:root:Requested phone number is not found 21212


3
'2'
None


In [211]:
# Close the connection to the Aerospike cluster
client.close()